In [13]:
import pandas as pd
import numpy as np
import datetime
import tensorflow as tf
import numpy as np
import metrics
import time
import useful_fns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import os

In [14]:
# hyper parameters:
# -FF
# num layers
# size of layers
# Batch Norm

# -LSTM
# num layers
# size of layers
# Batch Norm

# -v
# 
# -d
# softplus factor rho
# 
# -m
# posterior softplus factor rho
# prior std
# size of dense variational layer
# DistributionLambda scale

# -c
# posterior softplus factor rho
# prior std
# size of dense variational layer
# softplus factor rho

# args=  Namespace(Arch='FF', 
#                  num_layers=1,
#                  sizeof_layers=256,
#                  batch_norm=True,
#                  Ext='-c', 
#                  kl_anneal=0.1,
#                  rho_q=10,
#                  rho_op=0.25,
#                  prior_scale=0.1,
#                  sizeof_bnn=25,
#                  Batch_Size=64, 
#                  Epochs=5, 
#                  Gamma=14, 
#                  country='us', 
#                  early_stopping=False, 
#                  smooth=True,
#                 )

In [10]:
import importlib
import model
importlib.reload(model)
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

def run_model(kl_anneal, num_layers, prior_scale, rho_op, rho_q, sizeof_bnn, sizeof_layers):
    os.chdir('/home/mimorris/Projects/Neural-Network-Flu-Uncertainty/src')
    args = Namespace(Arch='FF', 
                 num_layers=int(num_layers),
                 sizeof_layers=int(sizeof_layers),
                 batch_norm=True,
                 Ext='-c', 
                 kl_anneal=float(kl_anneal),
                 rho_q=float(rho_q),
                 rho_op=float(rho_op),
                 prior_scale=float(prior_scale),
                 sizeof_bnn=int(sizeof_bnn),
                 Batch_Size=64, 
                 Epochs=100, 
                 Gamma=0, 
                 country='us', 
                 early_stopping=False, 
                 smooth=True,
                )
    
    data = data_builder_new(root="/home/mimorris/Datasets/Flu/", 
                            country = args.country)
    
    results = results_table(args)
    year=1
    try:
        tf.keras.backend.clear_session()
        x_train, y_train, x_val, y_val, x_test, y_test = data.build(test_year=2014+year, 
                                                                validation_year=2013+year, 
                                                                start_of_season = '-08-23', 
                                                                gamma=args.Gamma, 
                                                                lag=14, 
                                                                window_size=28,
                                                                smooth=args.smooth)

        NN = model.model_builder(x_train, y_train.values[:, np.newaxis], args=args)
        NN.fit(x_train, y_train.values[:, np.newaxis], verbose=False)

        val_pred = NN.predict(x_val, pd.DataFrame(index=y_val.index, data=y_val.values, columns=['T0']))
        test_pred = NN.predict(x_test, pd.DataFrame(index=y_test.index, data=y_test.values, columns=['T0']))

        val_pred = rescale(val_pred, data)
        test_pred = rescale(test_pred, data)
        results.update(test_pred, val_pred, year)
    except:
        return -100
        
    results.save('results/hyper_parameter_optimisation/', get_num())
    os.listdir('results/hyper_parameter_optimisation/')
    
    if not np.isnan( -results.val_metrics['Average']['MB Log']):
        return(results.val_metrics['Average']['MB Log'])
    else:
        return -100

In [11]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs

pbounds = {'kl_anneal': (0.1, 5.0),
           'num_layers': (1, 4), 
           'prior_scale':(0.01, 10), 
           'rho_op':(0.1, 10),
           'rho_q':(0.1, 10),
           'sizeof_bnn':(2, 128),
           'sizeof_layers': (16, 512)
          }
           
optimizer = BayesianOptimization(
    f=run_model,
    pbounds=pbounds,
    random_state=10,
    verbose=2
)           
# load_logs(optimizer, logs=["./logs.json"]);
# logger = JSONLogger(path="./logs.json")
# optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

optimizer.maximize(
    init_points=5,
    n_iter=1000,
)

|   iter    |  target   | kl_anneal | num_la... | prior_... |  rho_op   |   rho_q   | sizeof... | sizeof... |
-------------------------------------------------------------------------------------------------------------
|  1        | -2.856    |  3.879    |  1.062    |  6.34     |  7.513    |  5.035    |  30.32    |  114.2    |
|  2        | -2.369    |  3.827    |  1.507    |  0.8925   |  6.885    |  9.539    |  2.497    |  270.0    |
|  3        | -2.712    |  4.082    |  2.838    |  7.22     |  2.99     |  9.186    |  92.04    |  285.1    |
|  4        | -3.379    |  0.7966   |  2.12     |  6.745    |  4.474    |  4.397    |  79.84    |  270.5    |
|  5        | -2.455    |  3.287    |  2.803    |  8.054    |  5.264    |  9.096    |  42.22    |  60.87    |
|  6        | -2.054    |  3.765    |  1.305    |  3.43     |  5.78     |  7.289    |  4.389    |  269.4    |
|  7        | -2.266    |  4.967    |  2.262    |  5.367    |  1.505    |  5.345    |  6.708    |  265.8    |
|  8      

/home/mimorris/source~/.bashrc/envs/py3/lib/python3.9/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


|  774      | -4.641    |  0.1      |  1.0      |  10.0     |  0.1      |  10.0     |  7.027    |  429.3    |
|  775      | -4.758    |  5.0      |  1.0      |  0.01     |  0.1      |  10.0     |  79.84    |  124.6    |
|  776      | -4.632    |  5.0      |  4.0      |  10.0     |  0.1      |  10.0     |  50.7     |  389.7    |
|  777      | -4.757    |  0.1      |  4.0      |  0.01     |  0.1      |  10.0     |  2.0      |  331.9    |
|  778      | -2.085    |  0.1      |  1.0      |  10.0     |  10.0     |  10.0     |  27.03    |  83.04    |
|  779      | -4.758    |  0.1      |  4.0      |  0.01     |  0.1      |  10.0     |  96.24    |  257.3    |
|  780      | -2.183    |  2.694    |  2.037    |  0.2927   |  2.107    |  9.781    |  15.91    |  442.4    |
|  781      | -6.88     |  5.0      |  4.0      |  10.0     |  0.1      |  0.1      |  56.65    |  400.3    |
|  782      | -4.672    |  5.0      |  1.0      |  10.0     |  0.1      |  10.0     |  16.94    |  460.3    |
|  783    

/home/mimorris/source~/.bashrc/envs/py3/lib/python3.9/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


|  853      | -2.774    |  0.1      |  4.0      |  0.01     |  10.0     |  10.0     |  2.0      |  371.5    |
|  854      | -7.276    |  0.1      |  4.0      |  0.01     |  0.1      |  0.1      |  115.7    |  262.7    |
|  855      | -4.758    |  5.0      |  1.0      |  0.01     |  0.1      |  10.0     |  14.82    |  472.3    |
|  856      | -4.203    |  0.1      |  1.0      |  10.0     |  0.1      |  10.0     |  91.77    |  47.52    |
|  857      | -2.019    |  5.0      |  4.0      |  0.01     |  10.0     |  10.0     |  14.44    |  281.3    |
|  858      | -4.397    |  0.1      |  4.0      |  10.0     |  0.1      |  10.0     |  14.6     |  90.85    |
|  859      | -4.757    |  5.0      |  4.0      |  0.01     |  0.1      |  10.0     |  2.0      |  282.7    |
|  860      | -3.888    |  5.0      |  4.0      |  10.0     |  0.1      |  10.0     |  87.96    |  16.0     |
|  861      | -100.0    |  0.1      |  1.0      |  0.01     |  10.0     |  10.0     |  55.34    |  107.2    |
|  862    

In [12]:
# {'target': -1.5207621394968815,
#  'params': {'kl_anneal': 1.6660758296625586,
#   'num_layers': 2.278738014928133,
#   'prior_scale': 6.751488193351199,
#   'rho_op': 2.613560914002351,
#   'rho_q': 9.386220894044913,
#   'sizeof_bnn': 3.3659473914427593,
#   'sizeof_layers': 30.1614365856536}}

In [ ]:
# goal is -1.022